<a href="https://colab.research.google.com/github/FOLLGAD/dreaming-video/blob/main/Music_video_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [1]:
%%capture
! pip install stable_diffusion_videos[realesrgan]
! git config --global credential.helper store

## Authenticate with Hugging Face Hub

You have to be a registered user in 🤗 Hugging Face Hub, and you'll also need to use an access token for the code to work. For more information on access tokens, please refer to [this section of the documentation](https://huggingface.co/docs/hub/security-tokens).

In [2]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


### Load the Interface

This step will take a couple minutes the first time you run it.

In [3]:
import torch

from stable_diffusion_videos import StableDiffusionWalkPipeline, Interface

pipeline = StableDiffusionWalkPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16,
    revision="fp16",
    safety_checker=None,
).to("cuda")

interface = Interface(pipeline)

Downloading:   0%|          | 0.00/543 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/342 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.70k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/608M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/806 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.72G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/609 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/167M [00:00<?, ?B/s]

You have passed `None` for safety_checker to disable its functionality in <class 'stable_diffusion_videos.stable_diffusion_pipeline.StableDiffusionWalkPipeline'>. Note that this might lead to problems when using <class 'stable_diffusion_videos.stable_diffusion_pipeline.StableDiffusionWalkPipeline'> and is not recommended.
ftfy or spacy is not installed using BERT BasicTokenizer instead of ftfy.
/usr/local/lib/python3.8/dist-packages/gradio/blocks.py:717: UserWarning: api_name predict already exists, using predict_1
  warnings.warn(


In [4]:
#@title Connect to Google Drive to Save Outputs

#@markdown If you want to connect Google Drive, click the checkbox below and run this cell. You'll be prompted to authenticate.

#@markdown If you just want to save your outputs in this Colab session, don't worry about this cell

connect_google_drive = True #@param {type:"boolean"}

#@markdown Then, in the interface, use this path as the `output` in the Video tab to save your videos to Google Drive:

#@markdown > /content/gdrive/MyDrive/stable_diffusion_videos


if connect_google_drive:
    from google.colab import drive

    drive.mount('/content/gdrive')

Mounted at /content/gdrive


Walk! 🚶‍♀️

In [5]:
#@title Audio url
!pip install yt-dlp
music_video_url = "https://www.youtube.com/watch?v=hEWPmWprx7Y" #@param {type:"string"}

import yt_dlp
ydl = yt_dlp.YoutubeDL({
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
    }],
    'outtmpl': 'audio.mp3',
})
ydl.download([music_video_url])
"DONE"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.8 MB 28.1 MB/s 
     |████████████████████████████████| 193 kB 80.6 MB/s 
     |████████████████████████████████| 357 kB 72.0 MB/s 
     |████████████████████████████████| 2.3 MB 60.0 MB/s 
[youtube] hEWPmWprx7Y: Downloading webpage
[youtube] hEWPmWprx7Y: Downloading android player API JSON
[info] hEWPmWprx7Y: Downloading 1 format(s): 251
[download] Destination: audio.mp3
[download] 100% of    3.63MiB in 00:00:00 at 59.56MiB/s  
[ExtractAudio] Destination: audio.mp3
Deleting original file audio.orig.mp3 (pass -k to keep)


'DONE'

In [6]:
#@title Whisper it

!pip install git+https://github.com/openai/whisper.git
!whisper audio.mp3 --model medium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-_ljom507
  Running command git clone -q https://github.com/openai/whisper.git /tmp/pip-req-build-_ljom507
  Created wheel for whisper: filename=whisper-1.0-py3-none-any.whl size=1175239 sha256=ee6a5d31e46162a55595b40424693001a7be2e86328f10b152352064ad14587e
  Stored in directory: /tmp/pip-ephem-wheel-cache-6c1dnypk/wheels/a7/70/18/b7693c07b1d18b3dafb328f5d0496aa0d41a9c09ef332fd8e6
Successfully built whisper
100%|██████████████████████████████████████| 1.42G/1.42G [00:11<00:00, 138MiB/s]
tcmalloc: large alloc 1528012800 bytes == 0x98f6000 @  0x7f8365bb11e7 0x4d3280 0x5de162 0x60fd9f 0x5a9e9b 0x46eb22 0x616b6b 0x4f7ada 0x49ca7c 0x5d7c18 0x49ec69 0x55e571 0x5d7cf1 0x49ced5 0x5d7c18 0x49ec69 0x55e571 0x55ef23 0x642140 0x6421be 0x644688 0x644c2c 0x6776be 0x677889 0x7f83657aec87 0x5e0f1a
Detecting language using up to the fir

In [7]:
import os
def parse_timestamp(timestamp: str) -> float:
    h, m, s = timestamp.split(":")
    s, ms = s.split(",")
    return 3600 * int(h) + 60 * int(m) + int(s) + 0.001 * int(ms)
with open("audio.mp3.srt", "r") as f:
    content = f.read()
blocks = content.strip().split(os.linesep + os.linesep)
ls=[]
for block in blocks:
    lines = block.strip().split("\n")
    t1, t2 = lines[1].split("-->")
    start_time = parse_timestamp(t1.strip())
    end_time = parse_timestamp(t2.strip())
    text = " ".join(lines[2:])
    ls.append({
        "start": start_time,
        "end": end_time,
        "text": text,
    })

lyrics = ls
lyrics

[{'start': 1.0, 'end': 5.0, 'text': 'Take a pill and go to sleep'},
 {'start': 6.0, 'end': 10.0, 'text': "I'm chasing witches in the street"},
 {'start': 11.0, 'end': 15.0, 'text': "I'm the last page in your book"},
 {'start': 16.0, 'end': 20.0, 'text': "Can't write a song, only do hooks"},
 {'start': 21.0, 'end': 24.0, 'text': 'Watching horses in the fields'},
 {'start': 24.0, 'end': 29.0, 'text': 'The dragon rests in agony'},
 {'start': 30.0, 'end': 34.0, 'text': "When I'm afraid I'll lose my mind"},
 {'start': 35.0, 'end': 39.0, 'text': "It's fun, it happens all the time"},
 {'start': 40.0, 'end': 44.0, 'text': "When I'm afraid I'll lose my mind"},
 {'start': 45.0, 'end': 48.0, 'text': "It's fun, it happens all the time"},
 {'start': 48.0, 'end': 51.0, 'text': 'Isolation caved in'},
 {'start': 51.0, 'end': 55.0, 'text': 'I adore you, the sound of your skin'},
 {'start': 81.0, 'end': 85.0, 'text': "So many lies that I've found"},
 {'start': 86.0, 'end': 90.0, 'text': 'Blood, heaven, 

In [8]:
import random

newlyrics = [{'start': 0, 'end': 0, 'text':"Dark screen"}] + lyrics
newlyrics = newlyrics[:16]
midpoints = [(l['start'] + l['end']) / 2 for l in newlyrics] # use midpoints to make it good :)

def genPrompt(text):
    return f'{text}. Trending on Artstation'

prompts = [genPrompt(l['text']) for l in newlyrics] # lyrics as the prompts

audio_offsets = midpoints
fps = 2

# Convert seconds to frames
num_interpolation_steps = [(b-a) * fps for a, b in zip(audio_offsets, audio_offsets[1:])]
print(f"will do {num_interpolation_steps} int. steps")

randarray = [random.randint(1, 2656000) for i in midpoints]
print(randarray)

video_path = pipeline.walk(
    prompts=prompts,
    seeds=randarray,
    num_interpolation_steps=num_interpolation_steps,
    guidance_scale=7.0,
    height=512,                            # use multiples of 64
    width=512,                             # use multiples of 64
    audio_filepath='audio.mp3',  # Use your own file
    audio_start_sec=0,      # Start second of the provided audio
    fps=fps,                               # important to set yourself based on the num_interpolation_steps you defined
    batch_size=2,                          # increase until you go out of memory
    output_dir='./dreams',                 # Where images will be saved
    name=None,                             # Subdir of output dir. will be timestamp by default
)


will do [6.0, 10.0, 10.0, 10.0, 9.0, 8.0, 11.0, 10.0, 10.0, 9.0, 6.0, 7.0, 60.0, 10.0, 10.0] int. steps
[1207080, 758695, 2341458, 603901, 596197, 2011787, 1832430, 2354306, 190885, 621191, 69254, 668589, 2318786, 745344, 2544973, 2342151]


/usr/local/lib/python3.8/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


  0%|          | 0/51 [00:00<?, ?it/s]

  0%|          | 0/51 [00:00<?, ?it/s]

  0%|          | 0/51 [00:00<?, ?it/s]

...

  0%|          | 0/51 [00:00<?, ?it/s]

  0%|          | 0/51 [00:00<?, ?it/s]

  0%|          | 0/51 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.8/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


NameError: ignored

In [10]:
from IPython.display import HTML
from base64 import b64encode

def visualize_video_colab(video_path):
    mp4 = open(video_path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML("""
    <video width=400 controls>
        <source src="%s" type="video/mp4">
    </video>
    """ % data_url)
  
visualize_video_colab(video_path)

In [12]:
#@title Overlay lyrics on video
!ffmpeg -i "$video_path" -vf "subtitles=audio.mp3.srt" \
  -c:v libx264 -crf 20 \
  -c:a aac -b:a 192k \
  outvideo.mp4
!#mv outvideo.mp4 gdrive/MyDrive/dreamgony-agony-yung-lean.mp4

ffmpeg version 3.4.11-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-li

In [ ]:
 #@title or just launch the interface
interface.launch(debug=True, share=True)